In [1]:
import numpy as np

In [ ]:
# 一般化Lasso（Fused Lasso含む）を ADMM で解くデモコード
# 目的：最小化  (1/2)||Ax - b||_2^2 + C1 * ||D_Fus x||_1
#   ・D_Fus = [ I ; (C2/C1) * D ] により、L1（係数のスパース）と差分L1（平滑/区分一定）を同時に表現
#   ・ADMM の y = D_Fus x 導入により、y 更新は soft-threshold（prox）で閉じる
#
# 注意（警告/エラーを避けるため）：
#   - このスクリプト自身で numpy を import（モジュール内部の import は現在スコープへは来ない）
#   - matplotlib の使用は一般的な API のみ（非推奨引数は未使用）
#   - ベクトル・行列次元は整合するように D と D_Fus を構築
#   - hlines の y は配列として [0.0] を渡す（スカラーでも良いが配列の方が一貫）
from code8_8 import *  # generalized_lasso_ADMM（ADMM本体）
from code8_4 import *  # soft_thresholding（L1近接写像）
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib  # 日本語フォント設定（副作用で rcParams が適用される）

# -----------------------------
# データ生成（スパースかつ区分一定の真の信号）
# -----------------------------
np.random.seed(0)
n, m = 100, 500  # 変数次元 n、サンプル数 m（m>n で過剰決定）
A = np.random.randn(m, n)  # デザイン行列 A ∈ R^{m×n}
x_true = np.zeros(n)  # 真のパラメータ（疎 + 区分一定）
x_true[10:20] = 2.0  # 区間1：一定の正値
x_true[40:50] = -1.5  # 区間2：一定の負値
x_true[70:80] = 3.0  # 区間3：一定の正値
x_true[90:95] = -1.0  # 区間4：一定の負値
b = A @ x_true + np.random.randn(m) * 0.1  # ノイズ付き観測

# -----------------------------
# 正則化強度
#   C1: 係数に対する L1（スパース）
#   C2: 1階差分に対する L1（区分一定促進；Fused Lasso）
# -----------------------------
C1, C2 = 1.0, 10.0

# -----------------------------
# 差分行列 D の構築（前進差分）
#   D ∈ R^{(n-1)×n}, (Dx)_i = x_i - x_{i+1}
#   → ||Dx||_1 を小さくすると隣接差が0に近づき、区分一定を促す
# -----------------------------
D = np.eye(n) - np.eye(n, k=1)  # 対角と1つ右上に -1
D = D[:-1, :]  # 最終行はサイズ合わせで削除 → (n-1)×n

# -----------------------------
# 一般化 Lasso の D_Fus = [ I ; (C2/C1) * D ]
#   C1 * || [I; (C2/C1)D ] x ||_1 = C1||x||_1 + C2||Dx||_1
#   1つの ADMM で L1 と Fused を同時に扱えるように縦結合
# -----------------------------
D_Fus = np.vstack([np.eye(n), (C2 / C1) * D])  # 形状：(n + n - 1) × n

# -----------------------------
# 初期値設定
#   y は D_Fus x の次元に合わせる
# -----------------------------
x_0 = np.zeros(n)  # x の初期点
y_0 = np.zeros(D_Fus.shape[0])  # y の初期点（= D_Fus x の次元）
mu_0 = np.zeros(D_Fus.shape[0])  # μ の初期点（スケーリング乗数）

# -----------------------------
# ADMM 実行
#   ・prox_y は soft_thresholding を使用（L1 近接）
#   ・rho_k は初期のペナルティ係数（自動調整あり）
# -----------------------------
x, y, mu = generalized_lasso_ADMM(
    A=A,
    b=b,
    D=D_Fus,
    C=C1,
    x_k=x_0,
    y_k=y_0,
    prox_y=soft_thresholding,  # y ← prox_{(C/ρ)||·||_1}(·)
    rho_k=10.0,  # 初期 ρ
    max_iter=500,  # 反復上限
    eta=10,
    zeta1=2,
    zeta2=2,  # ρ のバランス調整パラメータ
    eps3=1e-3,
    eps4=1e-4,  # 停止判定（絶対/相対閾値）
)

# -----------------------------
# 可視化：推定された x を散布図で表示
# -----------------------------
fig, ax = plt.subplots(figsize=(8, 4.5))
ax.scatter(np.arange(n), x, s=18, label="推定係数")
ax.hlines(
    [0.0],
    xmin=-1,
    xmax=n,
    colors="blue",
    linestyles="dashed",
    linewidth=1.0,
    label="0軸",
)
ax.set_xlabel(r"要素番号 $i$", fontsize=14)
ax.set_ylabel(r"回帰係数 $x_i$", fontsize=14)
ax.set_xlim(-1, n)
ax.legend()
fig.tight_layout()
plt.show()